In [ ]:
print("This Code is Useful For Detecting Angles of the Parts Required for the project")

Import and Install Dependencies.    

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose


In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame=cap.read()
    cv2.imshow('Mediapipe feed',frame)
    if(cv2.waitKey(10) & 0xFF==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()


Make Detection.

In [36]:
def calculateAngle(a,b,c):
    a=np.array(a) # First
    b=np.array(b) # Mid
    c=np.array(c) # End

    x=np.arctan2(c[1]-b[1],c[0]-b[0])
    y=np.arctan2(a[1]-b[1],a[0]-b[0])
    radians=x-y
    angle=np.abs(radians*180/np.pi)
    if angle>180.0:
        angle=360-angle
    return angle


In [39]:
import cv2
import mediapipe as mp
import numpy as np


def calculateAngle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    x = np.arctan2(c[1] - b[1], c[0] - b[0])
    y = np.arctan2(a[1] - b[1], a[0] - b[0])
    radians = x - y
    angle = np.abs(radians * 180 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [41]:
import time
time.sleep(5)
state=0
count=0
# Initialize VideoCapture for the default camera
cap = cv2.VideoCapture(0)

# Setup MediaPipe Pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make Pose detection
        results = pose.process(image)

        # Convert image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

            angleTop = calculateAngle(shoulder, elbow, wrist)
            angleBottom = calculateAngle(shoulder, hip, knee)
            text="InCorrect Posture"
            
            ## First Detect Straight Posture.
            if state==0:
                
                if angleBottom>172 & angleBottom<=180:
                    state=1
                    
                else:
                    text
            elif state==1:
                if angleTop>=40 & angleTop<=120:
                    # Folded Hands Posture 
                    state=2
                    # Completed the state-> Moving to next state.
            
            elif state==2:
                ## Should Detect the back bending state.
                if angleTop>=175 &  (angleBottom>=160 & angleBottom<172):
                    state=3
            
            elif state==3:
                ## Detect a Forward Bending  State.
                if  angleBottom>=170 & angleBottom<=180 & angleTop>=50 & angleTop<=80:
                    state=0
                    ## Back to state -2
                    count+=1


            
            # Display angles in a box at the top left corner
            cv2.rectangle(image, (30, 30), (320, 90), (0, 0, 0), -1)
            
            cv.put

            cv2.putText(image, f'Top Angle: {angleTop:.2f} degrees', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Bottom Angle: {angleBottom:.2f} degrees', (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=3, circle_radius=2)
                                  )

        # Display the frame with landmarks
        cv2.imshow('MediaPipe feed', image)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the camera and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()
print("Hele")


Hele


In [ ]:
calculateAngle(shoulder,elbow,wrist)